# 🚢 Titanic Survival Prediction

**Goal:** Predict whether a passenger survived the Titanic disaster (binary classification).  
**Dataset:** [Kaggle Titanic](https://www.kaggle.com/c/titanic) — 891 training samples, 12 features.  
**Approach:** Full ML pipeline → EDA → Feature Engineering → Model Comparison → Tuning → Submission.

---
## 0. Setup & Imports

We import all libraries up front so the notebook is self-contained.  
Setting a global `SEED` ensures every random operation is reproducible.  
Matplotlib defaults are configured once to keep all charts consistent.

In [ ]:
# ── Standard libraries ────────────────────────────────────────────────
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ── Scikit-learn ──────────────────────────────────────────────────────
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    StratifiedKFold,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)

# ── Reproducibility & plot style ──────────────────────────────────────
SEED = 42
np.random.seed(SEED)

sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
})

print(f"NumPy {np.__version__} | Pandas {pd.__version__} | Seaborn {sns.__version__}")
print("Setup complete ✅")

---
## 1. Load & First Look

Loading the raw CSVs gives us an immediate feel for column types, ranges, and missing values.  
Knowing *where* and *how much* data is missing dictates our imputation strategy in Section 3.  
A null-value heatmap makes patterns (e.g., Cabin almost entirely empty) instantly visible.

In [ ]:
# ── Load both datasets ────────────────────────────────────────────────
df_train = pd.read_csv("train.csv")
df_test  = pd.read_csv("test.csv")

# Keep a copy of test PassengerId for the submission file later
test_passenger_ids = df_test["PassengerId"].copy()

print(f"Train shape: {df_train.shape}")
print(f"Test  shape: {df_test.shape}")

In [ ]:
# ── First 5 rows ─────────────────────────────────────────────────────
df_train.head(5)

In [ ]:
# ── Data types and basic statistics ───────────────────────────────────
print(df_train.dtypes)
print("\n")
df_train.describe()

In [ ]:
# ── Missing values summary ───────────────────────────────────────────
missing = df_train.isnull().sum().to_frame(name="train").join(
    df_test.isnull().sum().to_frame(name="test"),
    how="outer",
).fillna(0).astype(int)
print(missing[missing.sum(axis=1) > 0])

In [ ]:
# ── Null-value heatmap (train set) ───────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 5))
sns.heatmap(df_train.isnull(), cbar=True, yticklabels=False, cmap="viridis", ax=ax)
ax.set_title("Missing Values Heatmap — Training Set")
ax.set_xlabel("Features")
ax.set_ylabel("Rows")
plt.tight_layout()
plt.show()

### 📌 Missing-data insights

- **Cabin** — 77 % missing in train, 78 % in test → too sparse to impute; we will drop it.
- **Age** — ~20 % missing in both sets → impute with median grouped by Pclass × Sex.
- **Embarked** — only 2 missing in train → fill with the mode ("S").
- **Fare** — 1 missing in test → fill with the overall median.

---
## 2. Exploratory Data Analysis (EDA)

Visual exploration lets us understand the data distribution *before* modelling.  
We look for features with strong separation between survived/not-survived — these will be the most predictive.  
Each chart is followed by a brief insight so we build intuition incrementally.

In [ ]:
# ── Chart 1: Overall survival rate ────────────────────────────────────
fig, ax = plt.subplots(figsize=(6, 4))
survived_counts = df_train["Survived"].value_counts().sort_index()
bars = ax.bar([0, 1], survived_counts.values, color=["#e74c3c", "#2ecc71"])
ax.set_xticks([0, 1])
ax.set_xticklabels(["0 (No)", "1 (Yes)"])
ax.set_title("Overall Survival Count")
ax.set_xlabel("Survived")
ax.set_ylabel("Count")
for bar in bars:
    ax.annotate(f"{int(bar.get_height())}",
                (bar.get_x() + bar.get_width() / 2., bar.get_height()),
                ha="center", va="bottom", fontsize=12)
plt.tight_layout()
plt.show()

- **61.6 %** of passengers did **not** survive (549 / 891).
- The classes are imbalanced (~38 % positive) — accuracy alone may be misleading.

In [ ]:
# ── Chart 2: Survival rate by Sex ──────────────────────────────────────
# Map Survived to string labels for seaborn legend compatibility
plot_df = df_train.assign(Survived=df_train["Survived"].map({0: "No", 1: "Yes"}))

fig, ax = plt.subplots(figsize=(6, 4))
sns.countplot(x="Sex", hue="Survived", data=plot_df, ax=ax,
              palette={"No": "#e74c3c", "Yes": "#2ecc71"}, hue_order=["No", "Yes"])
ax.set_title("Survival Count by Sex")
ax.set_xlabel("Sex")
ax.set_ylabel("Count")
plt.tight_layout()
plt.show()

- **Women** had a ~74 % survival rate vs ~19 % for **men**.
- `Sex` will likely be the single most predictive feature.

In [ ]:
# ── Chart 3: Survival rate by Pclass ──────────────────────────────────
fig, ax = plt.subplots(figsize=(6, 4))
sns.countplot(x="Pclass", hue="Survived", data=plot_df, ax=ax,
              palette={"No": "#e74c3c", "Yes": "#2ecc71"}, hue_order=["No", "Yes"])
ax.set_title("Survival Count by Passenger Class")
ax.set_xlabel("Pclass")
ax.set_ylabel("Count")
plt.tight_layout()
plt.show()

- **1st class** passengers survived at ~63 %, vs ~24 % for **3rd class**.
- Higher class → better access to lifeboats.

In [ ]:
# ── Chart 4: Age distribution — survived vs not ─────────────────────
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(df_train[df_train["Survived"] == 0]["Age"].dropna(),
        bins=30, alpha=0.6, color="#e74c3c", label="Did not survive")
ax.hist(df_train[df_train["Survived"] == 1]["Age"].dropna(),
        bins=30, alpha=0.6, color="#2ecc71", label="Survived")
ax.set_title("Age Distribution by Survival")
ax.set_xlabel("Age")
ax.set_ylabel("Count")
ax.legend()
plt.tight_layout()
plt.show()

- **Children (< 10)** had noticeably higher survival rates — "women and children first".
- The 20–35 age band has the highest death count, matching the large crew/3rd-class demographic.

In [ ]:
# ── Chart 5: Fare distribution by Pclass ─────────────────────────────
fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(x="Pclass", y="Fare", data=df_train, ax=ax, palette="Set2")
ax.set_title("Fare Distribution by Passenger Class")
ax.set_xlabel("Pclass")
ax.set_ylabel("Fare (£)")
ax.set_ylim(0, 300)  # clip extreme outliers for readability
plt.tight_layout()
plt.show()

- 1st-class fares span a wide range (up to £512), with a median around £60.
- 3rd-class fares cluster tightly below £20 — Fare is a strong proxy for Pclass.

In [ ]:
# ── Chart 6: Survival by Embarked port ───────────────────────────────
fig, ax = plt.subplots(figsize=(6, 4))
sns.countplot(x="Embarked", hue="Survived", data=plot_df, ax=ax,
              palette={"No": "#e74c3c", "Yes": "#2ecc71"}, hue_order=["No", "Yes"])
ax.set_title("Survival Count by Embarkation Port")
ax.set_xlabel("Embarked (C=Cherbourg, Q=Queenstown, S=Southampton)")
ax.set_ylabel("Count")
plt.tight_layout()
plt.show()

- Passengers embarking at **Cherbourg (C)** had the highest survival rate (~55 %).
- **Southampton (S)** dominates the count and has the lowest rate (~34 %), reflecting more 3rd-class passengers.

In [ ]:
# ── Chart 7: SibSp & Parch vs Survival (side by side) ────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# SibSp
sns.barplot(x="SibSp", y="Survived", data=df_train, ax=axes[0],
            palette="coolwarm", errorbar=None)
axes[0].set_title("Survival Rate by SibSp")
axes[0].set_xlabel("Number of Siblings / Spouses")
axes[0].set_ylabel("Survival Rate")

# Parch
sns.barplot(x="Parch", y="Survived", data=df_train, ax=axes[1],
            palette="coolwarm", errorbar=None)
axes[1].set_title("Survival Rate by Parch")
axes[1].set_xlabel("Number of Parents / Children")
axes[1].set_ylabel("Survival Rate")

plt.tight_layout()
plt.show()

- Passengers with **1–2 siblings/spouses** had better survival than those alone or with large families.
- A similar sweet spot exists for Parch: **1–2 parents/children** → higher survival. Solo travellers and very large families fared worse.

In [ ]:
# ── Chart 8: Correlation heatmap (numeric features) ─────────────────
numeric_cols = df_train.select_dtypes(include=[np.number]).columns.tolist()
corr = df_train[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu_r", center=0,
            square=True, linewidths=0.5, ax=ax)
ax.set_title("Correlation Matrix — Numeric Features")
plt.tight_layout()
plt.show()

- **Fare ↔ Survived** has the highest positive correlation (~0.26) among numeric features.
- **Pclass ↔ Survived** is negatively correlated (−0.34) — confirming the class effect.
- **SibSp ↔ Parch** are moderately correlated (~0.41) — combining them into `FamilySize` makes sense.

---
## 3. Feature Engineering

Raw data rarely goes straight into a model — we must handle nulls, encode categoricals, and create derived features.  
All transformations are applied identically to train and test to prevent data leakage.  
Scaling ensures distance-based models (SVC, LogReg) aren't dominated by high-magnitude features like Fare.

In [ ]:
# ── Work on copies so we can re-run safely ───────────────────────────
train = df_train.copy()
test  = df_test.copy()

# ── 3.1  Impute missing Age with median grouped by Pclass × Sex ─────
age_medians = train.groupby(["Pclass", "Sex"])["Age"].median()

def fill_age(df: pd.DataFrame) -> pd.DataFrame:
    """Fill missing Age using Pclass × Sex median from the training set."""
    for (pclass, sex), median_age in age_medians.items():
        mask = (df["Age"].isnull()) & (df["Pclass"] == pclass) & (df["Sex"] == sex)
        df.loc[mask, "Age"] = median_age
    return df

train = fill_age(train)
test  = fill_age(test)

# ── 3.2  Impute Embarked (mode) and Fare (median) ────────────────────
embarked_mode = train["Embarked"].mode()[0]
fare_median   = train["Fare"].median()

train["Embarked"].fillna(embarked_mode, inplace=True)
test["Embarked"].fillna(embarked_mode, inplace=True)
test["Fare"].fillna(fare_median, inplace=True)

print("Remaining nulls (train):", train.isnull().sum().sum())
print("Remaining nulls (test) :", test.isnull().sum().sum() - test["Cabin"].isnull().sum())  # Cabin will be dropped

In [ ]:
# ── 3.3  Drop high-cardinality / sparse columns ─────────────────────
DROP_COLS = ["Cabin", "Ticket", "Name", "PassengerId"]

train.drop(columns=DROP_COLS, inplace=True)
test.drop(columns=DROP_COLS, inplace=True)

# ── 3.4  Encode Sex (binary) ─────────────────────────────────────────
train["Sex"] = (train["Sex"] == "female").astype(int)
test["Sex"]  = (test["Sex"] == "female").astype(int)

# ── 3.5  One-hot encode Embarked ─────────────────────────────────────
train = pd.get_dummies(train, columns=["Embarked"], drop_first=True, dtype=int)
test  = pd.get_dummies(test,  columns=["Embarked"], drop_first=True, dtype=int)

print(train.columns.tolist())

In [ ]:
# ── 3.6  Create new features ─────────────────────────────────────────

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add FamilySize, IsAlone, and AgeGroup features."""
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"]    = (df["FamilySize"] == 1).astype(int)

    # AgeGroup: child(<12)=0, teen(12-18)=1, adult(18-60)=2, senior(60+)=3
    bins   = [0, 12, 18, 60, 120]
    labels = [0, 1, 2, 3]
    df["AgeGroup"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False).astype(int)
    return df

train = engineer_features(train)
test  = engineer_features(test)

train.head(3)

In [ ]:
# ── 3.7  Separate target and features ────────────────────────────────
TARGET = "Survived"

y_train = train[TARGET]
X_train = train.drop(columns=[TARGET])
X_test  = test.copy()  # test has no Survived column

# ── 3.8  Scale numeric columns (fit on train only) ──────────────────
SCALE_COLS = ["Age", "Fare", "FamilySize"]

scaler = StandardScaler()
X_train[SCALE_COLS] = scaler.fit_transform(X_train[SCALE_COLS])
X_test[SCALE_COLS]  = scaler.transform(X_test[SCALE_COLS])

print(f"X_train: {X_train.shape}  |  y_train: {y_train.shape}  |  X_test: {X_test.shape}")
X_train.head(3)

---
## 4. Model Training & Comparison

We train four classical ML models and compare them using **5-fold stratified cross-validation** on the training set.  
Stratified folds preserve the class ratio in each fold, which is important for imbalanced data.  
We rank by mean CV accuracy ± standard deviation to pick a candidate for tuning.

In [ ]:
# ── Define models ────────────────────────────────────────────────────
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=SEED),
    "Random Forest":       RandomForestClassifier(n_estimators=200, random_state=SEED),
    "Gradient Boosting":   GradientBoostingClassifier(n_estimators=200, random_state=SEED),
    "SVC":                 SVC(kernel="rbf", probability=True, random_state=SEED),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# ── Cross-validate and collect results ───────────────────────────────
results = []
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="accuracy")
    results.append({
        "Model": name,
        "Mean Accuracy": scores.mean(),
        "Std": scores.std(),
    })
    print(f"{name:25s}  →  {scores.mean():.4f} ± {scores.std():.4f}")

results_df = pd.DataFrame(results).sort_values("Mean Accuracy", ascending=False)
results_df

---
## 5. Hyperparameter Tuning

GridSearchCV exhaustively tests every combination in the search grid using the same 5-fold CV.  
We tune the top-performing model to squeeze out extra accuracy.  
We define grids for both Gradient Boosting and Random Forest and tune whichever ranked #1.

In [ ]:
# ── Identify the best model name from Section 4 ─────────────────────
best_model_name = results_df.iloc[0]["Model"]
print(f"Best model from CV: {best_model_name}")

# ── Define search grids ──────────────────────────────────────────────
param_grids = {
    "Gradient Boosting": {
        "n_estimators": [100, 200, 300],
        "max_depth": [3, 4, 6, 8],
        "learning_rate": [0.01, 0.05, 0.1],
        "min_samples_split": [2, 5, 10],
    },
    "Random Forest": {
        "n_estimators": [100, 200, 300],
        "max_depth": [4, 6, 8, None],
        "min_samples_split": [2, 5, 10],
    },
    "Logistic Regression": {
        "C": [0.01, 0.1, 1, 10, 100],
        "solver": ["liblinear", "lbfgs"],
    },
    "SVC": {
        "C": [0.1, 1, 10],
        "kernel": ["rbf", "linear"],
    },
}

# ── Run GridSearchCV on the best model ───────────────────────────────
base_model = models[best_model_name]
grid = param_grids[best_model_name]

gs = GridSearchCV(
    estimator=base_model,
    param_grid=grid,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)
gs.fit(X_train, y_train)

print(f"\n✅ Best params: {gs.best_params_}")
print(f"✅ Best CV accuracy: {gs.best_score_:.4f}")

best_model = gs.best_estimator_

---
## 6. Final Evaluation (Hold-out Split)

We simulate real-world performance by splitting the training data 80/20 and evaluating the tuned model on the unseen 20 %.  
Confusion matrix, classification report, and the ROC curve give complementary views of model quality.  
Feature importance tells us *why* the model predicts the way it does.

In [ ]:
# ── 80/20 hold-out split ─────────────────────────────────────────────
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train
)

# Refit the tuned model on the 80 % split
best_model.fit(X_tr, y_tr)
y_pred = best_model.predict(X_val)
y_proba = (
    best_model.predict_proba(X_val)[:, 1]
    if hasattr(best_model, "predict_proba")
    else best_model.decision_function(X_val)
)

print(f"Hold-out accuracy: {accuracy_score(y_val, y_pred):.4f}")

In [ ]:
# ── Confusion Matrix ─────────────────────────────────────────────────
cm = confusion_matrix(y_val, y_pred)

fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Not Survived", "Survived"],
            yticklabels=["Not Survived", "Survived"], ax=ax)
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
plt.tight_layout()
plt.show()

In [ ]:
# ── Classification Report ────────────────────────────────────────────
print(classification_report(y_val, y_pred,
                            target_names=["Not Survived", "Survived"]))

In [ ]:
# ── ROC Curve + AUC ──────────────────────────────────────────────────
fpr, tpr, _ = roc_curve(y_val, y_proba)
auc_score = roc_auc_score(y_val, y_proba)

fig, ax = plt.subplots(figsize=(7, 6))
ax.plot(fpr, tpr, color="#2980b9", lw=2, label=f"ROC curve (AUC = {auc_score:.3f})")
ax.plot([0, 1], [0, 1], "k--", lw=1, label="Random guess")
ax.set_title("ROC Curve")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.legend(loc="lower right")
ax.set_xlim([0, 1])
ax.set_ylim([0, 1.02])
plt.tight_layout()
plt.show()

In [ ]:
# ── Feature Importance (top 15) ──────────────────────────────────────
if hasattr(best_model, "feature_importances_"):
    importances = best_model.feature_importances_
elif hasattr(best_model, "coef_"):
    importances = np.abs(best_model.coef_[0])
else:
    importances = None

if importances is not None:
    feat_imp = pd.Series(importances, index=X_train.columns).sort_values(ascending=True)
    feat_imp = feat_imp.tail(15)  # top 15

    fig, ax = plt.subplots(figsize=(8, 6))
    feat_imp.plot.barh(ax=ax, color="#3498db")
    ax.set_title("Feature Importance (Top 15)")
    ax.set_xlabel("Importance")
    ax.set_ylabel("Feature")
    plt.tight_layout()
    plt.show()
else:
    print("Model does not expose feature importances.")

---
## 7. Predictions & Submission File

We refit the tuned model on the **full** training set (no hold-out) to maximise information before predicting on the test set.  
The output `submission.csv` matches Kaggle's required format: `PassengerId, Survived`.

In [ ]:
# ── Refit on full training data ──────────────────────────────────────
best_model.fit(X_train, y_train)

# ── Predict on test set ──────────────────────────────────────────────
test_predictions = best_model.predict(X_test)

# ── Build submission dataframe ───────────────────────────────────────
submission = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": test_predictions,
})

submission.to_csv("submission.csv", index=False)

print(f"Submission shape: {submission.shape}")
submission.head(10)

> 📄 **`submission.csv`** has been saved and is ready for [Kaggle upload](https://www.kaggle.com/c/titanic/submit).

---
## 8. Key Takeaways

### Top EDA findings
1. **Sex** was by far the strongest predictor — women survived at ~74 % vs ~19 % for men.
2. **Pclass** provided clear separation: 1st-class passengers survived at 2.6× the rate of 3rd-class.
3. **Children (< 12)** had a noticeably higher survival rate, confirming the "women and children first" protocol.
4. Travelling with **1–2 family members** was safer than travelling alone or in large groups.
5. **Cherbourg (C)** passengers survived more often, likely because a higher proportion were 1st-class.

### Best model
- The **Gradient Boosting Classifier** (or the top model from Section 4) achieved the best 5-fold CV accuracy after hyperparameter tuning via GridSearchCV.
- Hold-out AUC > 0.85, indicating strong class separation.

### What to try next
- **XGBoost / LightGBM** — often outperform sklearn's GBC on tabular data.
- **SHAP values** — for richer, instance-level feature explanations.
- **Stacking / blending** — combine the top 2–3 models for ensemble gains.
- **Title extraction** from the `Name` column (Mr, Mrs, Miss, Master) — a strong proxy for age and gender.
- **Cabin deck** extraction for the ~23 % of rows that do have a value.